In [1]:
import pandas as pd
import os

# LOADING IN THE DATA

In [2]:
#Name of the folders
folders = ['A','B','C','D','del','E','F','G','H','I','J','K','L','M','N','nothing','O', 'P','Q','R','S','space','T','U','V','W','X','Y','Z'] 

#The labels the folders should have
labels = ['A','B','C','D','del','E','F','G','H','I','J','K','L','M','N','nothing','O', 'P','Q','R','S','space','T','U','V','W','X','Y','Z']

Fseries=pd.Series(folders, name='folder')
Lseries=pd.Series(labels, name='labels')

#Making the 
df=pd.concat([Fseries, Lseries], axis=1) 
print (df.head(10))

  folder labels
0      A      A
1      B      B
2      C      C
3      D      D
4    del    del
5      E      E
6      F      F


In [6]:
#Main directory where all the images are:
sdir= r"dataset\asl_alphabet_train\asl_alphabet_train"
filepaths=[]
labels=[]

#List of class sub directories
class_list=os.listdir(sdir)

for klass in class_list: # iterate over the class subdirectories
    class_path=os.path.join(sdir,klass)   # path to class sub directory 
    for i in range(len(df)):  # iterate through the data set      
        if str(df['folder'].iloc[i] )== klass:  #convert folder name to a string and compare to current klass          
            label=df['labels'].iloc[i] # get the associated label 
            flist=os.listdir(class_path) # get a list of all the files in the klass sub directory
            for f in flist: # iterate through the list of files
                fpath=os.path.join(class_path,f) # get the full path to the file
                filepaths.append(fpath) # append the full file path
                labels.append(str(label))    # append the label as a string 
                
Fseries=pd.Series(filepaths, name='filepaths')
Lseries=pd.Series(labels, name='labels')
data_df=pd.concat([Fseries, Lseries], axis=1) # create data frame with columns filepaths, labels
print(data_df.head(10))
print(data_df.tail(10))

                                           filepaths labels
0  dataset\asl_alphabet_train\asl_alphabet_train\...      A
1  dataset\asl_alphabet_train\asl_alphabet_train\...      A
2  dataset\asl_alphabet_train\asl_alphabet_train\...      A
3  dataset\asl_alphabet_train\asl_alphabet_train\...      A
4  dataset\asl_alphabet_train\asl_alphabet_train\...      A
5  dataset\asl_alphabet_train\asl_alphabet_train\...      A
6  dataset\asl_alphabet_train\asl_alphabet_train\...      A
7  dataset\asl_alphabet_train\asl_alphabet_train\...      A
8  dataset\asl_alphabet_train\asl_alphabet_train\...      A
9  dataset\asl_alphabet_train\asl_alphabet_train\...      A
                                               filepaths labels
86990  dataset\asl_alphabet_train\asl_alphabet_train\...      Z
86991  dataset\asl_alphabet_train\asl_alphabet_train\...      Z
86992  dataset\asl_alphabet_train\asl_alphabet_train\...      Z
86993  dataset\asl_alphabet_train\asl_alphabet_train\...      Z
86994  dataset\asl_a